In [ ]:
import pandas as pd

from gait_pipeline import make_dataset, topological_transform

from pre_process import find_period_acf


directory = 'data/'

In [ ]:
# Constructs a random sample of data by picking random windows of the subjects walking. Using period 
full_dataset = {}
full_entropies = {}

for sub_num in range(1,19):
    try:
        sub_dataset = make_dataset(f'{sub_num}', directory,sample_size = 150)
        full_dataset[sub_num] = sub_dataset
    except(ValueError):
        continue

for key in list(full_dataset.keys()):
    full_entropies[key] = topological_transform(full_dataset[key])

In [ ]:

entropy_period= []

for subject in full_entropies.keys():
    for measurement, dataset in zip(full_entropies[subject],full_dataset[subject]):
        H0, H1 = measurement[0],measurement[1]
        period = find_period_acf(dataset, 150)
        entropy_period.append([subject, H0,H1,period])

In [ ]:
entropy_df = pd.DataFrame(entropy_period)

In [ ]:
entropy_df.columns = ['subject', 'H0_Entropy', 'H1_Entropy','Period']

In [ ]:
from sklearn.model_selection import train_test_split

X = entropy_df.drop('subject', axis = 1)
y = entropy_df['subject']

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=10)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))